In [4]:
#-*- coding: utf-8 -*-

import requests
from lxml import html
import re
import csv
import os

page = requests.get('http://v1.syndication.nhschoices.nhs.uk/organisations/gppractices/atoz?apikey=JROLOVFO')
tree = html.fromstring(page.content)
links = tree.xpath('//li/a/@href')

if os.path.isfile("gp_details.csv") == True:
    os.remove("gp_details.csv")


#     PODSTAWOWA FUNKCJA POZWALAJĄCA NA ŚCIĄGNIĘCIE WŁAŚCIWYCH INFORMACJI Z PODSTRON O GP PRACTICES
def kielbaska (one_gp_link):
    pytania = ['telephone?', ' want one?', 'd respect?', 'treatment?', 'ning hours']
    kielbasa = []
    gp_page = requests.get(one_gp_link)
    tree4 = html.fromstring(gp_page.content)
    odpowiedzi = tree4.xpath('//span[@class="value" or @class="minValue" or @class="maxValue" or @class="numberOfRatings"]/text()')
    odpowiedzi = [re.sub(' +','', x.replace('\r\n', '')) for x in odpowiedzi]
    if len(odpowiedzi) == 20:
            kielbasa.extend(odpowiedzi)
    elif len(odpowiedzi) > 0:
#         print 'UWAGA: niestandardowa liczba odpowiedzi na pytania ankietowe: ' + str(len(odpowiedzi))
        zapytania = tree4.xpath('//div[@class="questionText"]/text()')
        podane_odpowiedzi = []
        for x in zapytania:
            x = re.sub(' +',' ', x.replace('\r\n', ''))
            if x[-10:] in pytania:
                starter = pytania.index(x[-10:])*4
                koniec = starter + 4
                podane_odpowiedzi.extend(range(starter,koniec))
            else:
                print False
                print 'nieznane pytanie: ' + str(x)
        pakunek = []
        licznik = 0
        for x in range(20):
            if x in podane_odpowiedzi:
                pakunek.append(odpowiedzi[licznik])
                licznik += 1
            else:
                pakunek.append('')
        kielbasa.extend(pakunek)
    else:
        kielbasa.extend(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''])

    tempik = gp_link.find("?apikey")
#   scraping z podstrony "overview"
    overview_link = gp_link[:tempik] + '/overview' + gp_link[tempik:]
    overview_page = requests.get(overview_link)
    tree_overview = html.fromstring(overview_page.content)

    overv1 = tree_overview.xpath('//div[@id="MainContent"]//dd[1]/text()')
    overv2 = tree_overview.xpath('//div[@id="MainContent"]//dd[2]/text()')
    kielbasa.append(overv1[0])
    kielbasa.append(overv2[0])
    kielbasa.append(overv1[1])
    kielbasa.append(overv2[1])
    
    overv2_2 = tree_overview.xpath('//div[@id="MainContent"]//*[text()="GPs at this practice:"]/following-sibling::dd[1]/text()')
    if len(overv2_2) != 0:
        lekarze = re.sub(' +',' ', overv2_2[0].replace('\r\n', ''))
        kielbasa.append(lekarze[1:-1])
    else: 
        kielbasa.append('')
    overv3 = tree_overview.xpath('//ul[@class="address"]//li/text()')
    kielbasa.extend(overv3[-3:])
    overv4 = tree_overview.xpath('//div[@id="MainContent"]//h2/text()')
    nazwa = overv4[0]
    try:
        nazwa.decode('ascii')
    except UnicodeEncodeError:
        nazwa = re.sub(u"\u2013", "-", overv4[0])
        nazwa2 = re.sub(u"\u2019", "'", nazwa)
        kielbasa.insert(0,nazwa2[:-11])
    else:
        kielbasa.insert(0,nazwa[:-11])
    info_overv = tree_overview.xpath('//div[@id="MainContent"]/descendant::text()')
    overv5 = tree_overview.xpath('//div[@id="MainContent"]//p/text()')
    obiekt = []
    for x in overv5:
        if x[:6] == 'Last v':
            obiekt.append(x[17:])
    if len(obiekt) == 2:
        kielbasa.extend(obiekt)
    elif len(obiekt) < 2:
        if len(obiekt) == 1:
            kielbasa.extend(obiekt)
            kielbasa.append('')
        else:
            kielbasa.append('')
            kielbasa.append('')
    else:
        print "UWAGA. Liczba wpisów 'last verified' jest większa niż 2"
#     TUTAJ TRZEBA COŚ PRZEROBIĆ ŻEBY PRAWIDŁOWO IMIONA I NAZWISKA LEKARZY SIĘ ZAPISYWAŁY

#     overv6 = tree_overview.xpath('//div[@id="MainContent"]//dd[1]//ul//li/text()')
    overv6 = tree_overview.xpath('//div[@id="MainContent"]//*[text()="Doctors:"]/following-sibling::dd[1]//ul//li/text()')
    overv6_mod = []
    for item in overv6:
        if item != 'Closed':
            overv6_mod.append(item)
    kielbasa.append(overv6_mod)
    overv7 = tree_overview.xpath('//div[@id="MainContent"]//*[text()="Is Electronic Prescription Service (EPS) available:"]/following-sibling::dd[1]/text()')
    kielbasa.extend(overv7)
    overv8 = tree_overview.xpath('//div[@id="MainContent"]//h2[1]/following-sibling::dl[1]//*/text()')
    overv8 = [re.sub(' +','', x.replace('\r\n', '')) for x in overv8]
#     print
#     print 'kiełbasa: ' + kielbasa[0]

    r_times = 0
    s_times = 0
#     out_hours = 0
    for aa, xx in enumerate(overv8):
        if len(xx) > 0:
            if xx[:10] == "Receptiont":
                r_times = aa
            elif xx[:10] == "Surgerytim":
                s_times = aa
#             elif xx[:10] == "Outofhours":
#                 out_hours = aa
#     print
#     print 'overv8'
#     print overv8
#     print 's_times'
#     print s_times
#     print 'r_times'
#     print r_times
    recep_list = []
    surg_list = []
    for aa, xx in enumerate(overv8):
        if aa > r_times and aa < s_times:
            if len(xx) > 0:
                recep_list.append(xx)
        elif aa > s_times:
            if len(xx) > 0:
                surg_list.append(xx)        
#     for aa, xx in enumerate(overv8):
#         if out_hours != 0:
#             if aa > r_times and aa < s_times:
#                 if len(xx) > 0:
#                     recep_list.append(xx)
#             elif aa > s_times and aa < out_hours:
#                 if len(xx) > 0:
#                     surg_list.append(xx)
#         else:
#             if aa > r_times and aa < s_times:
#                 if len(xx) > 0:
#                     recep_list.append(xx)
#             elif aa > s_times:
#                 if len(xx) > 0:
#                     surg_list.append(xx)
#     print 'recep_list'
#     print recep_list
#     print 'surg_list'
#     print surg_list
    if r_times != 0:
        recep_times = []
        week_recep = []
        sun = range(recep_list.index("Sunday")+1,recep_list.index("Monday"))
        mon = range(recep_list.index("Monday")+1,recep_list.index("Tuesday"))
        tue = range(recep_list.index("Tuesday")+1,recep_list.index("Wednesday"))
        wed = range(recep_list.index("Wednesday")+1,recep_list.index("Thursday"))
        thu = range(recep_list.index("Thursday")+1,recep_list.index("Friday"))
        fri = range(recep_list.index("Friday")+1,recep_list.index("Saturday"))
        sat = range(recep_list.index("Saturday")+1,len(recep_list))
        week_recep.append(sun)
        week_recep.append(mon)
        week_recep.append(tue)
        week_recep.append(wed)
        week_recep.append(thu)
        week_recep.append(fri)
        week_recep.append(sat)
        for x in week_recep:
            temp = []
            for item in x:
                temp.append(recep_list[item])
            recep_times.append(temp)
        kielbasa.extend(recep_times)
    else:
        kielbasa.extend(['','','','','','',''])
    if s_times != 0:
        surg_list2 = []
        maksimo = len(surg_list)
        for aa, xx in enumerate(surg_list):
            if aa <= surg_list.index('Saturday'):
                surg_list2.append(xx)
            else:
                if xx[0] in ['0','1','2','C'] and aa <= maksimo:
                    surg_list2.append(xx)
                else:
                    maksimo = aa
        week_surg = []
        surg_times = []
        sun_s = range(surg_list2.index("Sunday")+1,surg_list2.index("Monday"))
        mon_s = range(surg_list2.index("Monday")+1,surg_list2.index("Tuesday"))
        tue_s = range(surg_list2.index("Tuesday")+1,surg_list2.index("Wednesday"))
        wed_s = range(surg_list2.index("Wednesday")+1,surg_list2.index("Thursday"))
        thu_s = range(surg_list2.index("Thursday")+1,surg_list2.index("Friday"))
        fri_s = range(surg_list2.index("Friday")+1,surg_list2.index("Saturday"))
        sat_s = range(surg_list2.index("Saturday")+1,len(surg_list2))
        week_surg.append(sun_s)
        week_surg.append(mon_s)
        week_surg.append(tue_s)
        week_surg.append(wed_s)
        week_surg.append(thu_s)
        week_surg.append(fri_s)
        week_surg.append(sat_s)
        for x in week_surg:
            temp = []
            for item in x:
                temp.append(surg_list2[item])
            surg_times.append(temp)
        kielbasa.extend(surg_times)
    else:
        kielbasa.extend(['','','','','','',''])
    overv9 = tree_overview.xpath('//div[@id="MainContent"]//dl//*[text()="Parent organisation:"]/following-sibling::dd[1]//a/text()')
    kielbasa.extend(overv9)
#   scraping z podstrony "Services"
    services_link = gp_link[:tempik] + '/services' + gp_link[tempik:]
    services_page = requests.get(services_link)
    tree_services = html.fromstring(services_page.content)
    serv1 = tree_services.xpath('''//div[@id="MainContent"]//ul[@class="services"]//li//dd//a/text()''')
    kielbasa.append(serv1)
#   scraping z podstrony "Facilities"
    facilities_link = gp_link[:tempik] + '/facilities' + gp_link[tempik:]
    facilities_page = requests.get(facilities_link)
    tree_facilities = html.fromstring(facilities_page.content)
    facil1 = tree_facilities.xpath('//div[@id="MainContent"]//dl//dd//ul//li//dl//dd//ul//li/text()')
    facils = []
    for x in facil1:
        if x[-1] == 's':
            facils.append(x[:-6])
    kielbasa.append(facils)
#   scraping z podstrony "treatment indicators"
    treatments_link = gp_link[:tempik] + '/indicators/treatment' + gp_link[tempik:]
    treatments_page = requests.get(treatments_link)
    tree_treatments = html.fromstring(treatments_page.content)
    treat1 = tree_treatments.xpath('//div[@id="MainContent"]//ul[@id="links"]//li//a/text()')
    treat1 = [re.sub(' +',' ', x.replace('\r\n', '')) for x in treat1]
    l1 = [a.start() for a in re.finditer(' - ', treat1[1])]
    l2 = [a.start() for a in re.finditer(' - ', treat1[2])]
    l3 = [a.start() for a in re.finditer(' - ', treat1[4])]
    cut_off = 0
    for x in range(10):
        try:
            if l1[x] == l2[x] and l1[x] == l3[x]:
                cut_off = l1[x] + 3
            else:
                pass
        except IndexError:
            pass
    treatments = []
    for row in treat1:
        treatments.append(row[cut_off:])
    kielbasa.append(treatments)
    return kielbasa
# #                     scraping z podstrony "staff"
#                     staff_link = gp_link[:tempik] + '/staff' + gp_link[tempik:]
#                     staff_page = requests.get(staff_link)
#                     tree_staff = html.fromstring(staff_page.content)
#                     print staff_page.content
# #                 OGÓLNE INFORMACJE O PRACOWNIKACH
#                     staff1 = tree_staff.xpath('//div[@id="MainContent"]//ul//li/text()')

# #                 KATEGORIE PRACOWNIKÓW
#                     staff2 = tree_staff.xpath('//div[@id="MainContent"]//dl[2]//dt/text()')
    
#                     staff3 = tree_staff.xpath('//div[@id="MainContent"]//dl[2]//dd//ul//li/text()')
#                     staff_list = []
#                     for cat in staff2: 
                
# #                     scraping z podstrony "core indicators"
#                     core_link = gp_link[:tempik] + '/indicators/core' + gp_link[tempik:]
#                     core_page = requests.get(core_link)
#                     tree_core = html.fromstring(core_page.content)
# #                     core1 = tree_core.xpath('//div[@id="MainContent"]//ul//li//dl//*[text()="Value:"]/following::*[1]/text()')
#                     core1 = tree_core.xpath('//div[@id="MainContent"]//ul//li/node()')

#                     print kielbasa



with open('gp_details.csv', 'ab') as dok:
    writer = csv.writer(dok, delimiter=',')
    row0 = ['gp_name', 'telph_access_avg', 'telph_access_min', 'telph_access_max', 
            'telph_access_count', 'preferred_apptmnt_time_avg', 'preferred_apptmnt_time_min',
            'preferred_apptmnt_time_max', 'preferred_apptmnt_time_count', 'dignity_avg',
            'dignity_min', 'dignity_max', 'dignity_count', 'involve_dec_avg', 'involve_dec_min',
            'involve_dec_max', 'involve_dec_count', 'up_to_date_info_avg', 'up_to_date_info_min',
            'up_to_date_info_max', 'up_to_date_info_count', "index_letter", "ods", "lon", 'lat',
            'doctor_gender_ratio', 'town', 'county', 'postcode', 'last_verified_1', 'last_verified_2',
            'doctor_names', 'ePrescription_available', "reception_sun", 'reception_mon',
            'reception_tue', 'reception_wed', 'reception_thu', 'reception_fri', 'reception_sat',
            'surgery_sun', 'surgery_mon', 'surgery_tue', 'surgery_wed', 'surgery_thu',
            'surgery_fri', 'surgery_sat', 'parent_org', 'services', 'facilities',
            'treatment_indicators'
           ]
    writer.writerow(row0)
    for indeks, link in enumerate(links):
        if indeks < 8 and indeks > 5:
            print "zakładka: " + link[-19:-15]
            gp_list_page = requests.get(link)
            tree2 = html.fromstring(gp_list_page.content)
            last_page = tree2.xpath("//a[@id='lastPage']/@href")
            if len(last_page) == 1:
                last_page = str(last_page[0])
                temp = last_page.find("page=")
                nr = int(last_page[temp+5:])
                strony = range(1,nr+1)
                for page_nr in strony:
                    if page_nr > -1:
                        print "strona numer: " + str(page_nr)
                        page_link = link + '&page=' + str(page_nr)
                        gp_list_page2 = requests.get(page_link)
                        tree3 = html.fromstring(gp_list_page2.content)
                        gp_links = tree3.xpath('//li/a/@href')
                        for iii, gp_link in enumerate(gp_links):
                            if iii > -1:
                                wyniczek = kielbaska(gp_link)
                                writer.writerow(wyniczek)
                                if len(wyniczek) != 51:
                                    print 'UWAGA - nieprawidłowa długość wpisu dla GP: ' + str(wyniczek[0])
            elif len(last_page) == 0:
                gp_links = tree2.xpath('//li/a/@href')
                for iii, gp_link in enumerate(gp_links):
                    if iii > -1:
                        wyniczek = kielbaska(gp_link)
                        writer.writerow(wyniczek)
                        if len(wyniczek) != 51:
                            print 'UWAGA - nieprawidłowa długość wpisu dla GP: ' + str(wyniczek[0])
        else:
            print "UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe"
            


print
print "koniec"

UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
zakładka: z/f?
strona numer: 1
strona numer: 2
strona numer: 3
strona numer: 4
strona numer: 5
strona numer: 6
strona numer: 7
strona numer: 8
strona numer: 9
zakładka: z/g?
strona numer: 1
strona numer: 2
strona numer: 3
strona numer: 4
strona numer: 5
strona numer: 6
strona numer: 7
strona numer: 8
strona numer: 9
strona numer: 10
strona numer: 11
strona numer: 12
strona numer: 13
strona numer: 14
strona numer: 15
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana niż pozostałe
UWAGA! Jedna ze stron NHS Choices jest inaczej sformatowana ni

In [14]:
# data="St Catherine’s Surgery"
# udata=data.decode("utf-8")
# print udata
# asciidata=udata.encode("ascii","xmlcharrefreplace")
# print asciidata

mystring = "St Catherine’s Surgery"

try:
    mystring.decode('ascii')
except UnicodeDecodeError:
    print "it was not a ascii-encoded unicode string"
else:
    print "It may have been an ascii-encoded unicode string"


It may have been an ascii-encoded unicode string
